In [18]:
from load_data import load_data

cityc = load_data('POIdata_cityC.csv')
cityc

,x,y,category,POI_count
0,1,35,48,1
1,1,38,48,1
2,1,45,48,1
3,1,45,47,1
4,1,108,46,1
...,...,...,...,...
39059,187,200,81,1
39060,187,200,48,1
39061,188,199,63,1
39062,188,200,73,1


In [ ]:
# The headers of city C are: x, y, category, POI_count
from load_data import load_data

baskets = []

for city in 'B', 'C', 'D':
    raw = load_data(f'POIdata_city{city}.csv')
    groupd = raw.groupby(['x', 'y'], as_index=False).agg({'category': list})
    baskets.extend(groupd['category'].tolist())

[[48],
 [48],
 [48, 47],
 [46],
 [74, 45],
 [65],
 [65],
 [81],
 [43],
 [75, 79, 41],
 [75],
 [60, 54, 48, 75, 63],
 [73, 47],
 [51,
  18,
  48,
  47,
  57,
  79,
  36,
  54,
  73,
  5,
  59,
  56,
  10,
  3,
  58,
  81,
  38,
  21,
  66,
  69,
  26,
  46],
 [47, 48, 59, 66, 61, 55, 53, 43, 39, 51, 63, 33, 60, 36, 76],
 [67, 55, 48, 47, 43],
 [60, 48],
 [43, 10],
 [73],
 [47],
 [48],
 [80, 41, 60],
 [76, 84, 60, 48],
 [65],
 [75],
 [75],
 [20, 55, 84],
 [43],
 [79, 69, 9, 81, 74, 63, 82, 73, 48, 40, 60],
 [51, 54, 73, 63, 74, 82, 56, 36, 76, 9, 84, 81, 69, 48, 18, 47, 66],
 [63],
 [84],
 [75],
 [63],
 [48],
 [48],
 [48],
 [48, 47],
 [48, 3, 31],
 [45, 1],
 [31, 45],
 [60, 67],
 [76],
 [79],
 [18, 47, 31],
 [48],
 [80, 48, 75, 81, 38],
 [47,
  59,
  48,
  60,
  79,
  81,
  54,
  76,
  85,
  56,
  7,
  36,
  55,
  67,
  1,
  69,
  73,
  18,
  51,
  58,
  5,
  66],
 [47, 81, 66, 54, 56],
 [20],
 [21, 65, 4, 38, 18, 48, 63, 70, 66],
 [47],
 [60],
 [75, 60],
 [79],
 [81, 79, 76, 41],
 [5, 7

In [ ]:
import numpy as np


def generate_freq_itemsets(binary_matrix, unique_items, minsup):
    num_items = len(unique_items)
    freq_itemsets = {}
    
    # Generate frequent 1-itemsets
    item_counts = np.sum(binary_matrix, axis=0)
    freq_1 = np.where(item_counts >= minsup)[0]
    freq_itemsets[1] = [frozenset([unique_items[i]]) for i in freq_1]
    print(f"Generated {len(freq_itemsets[1])} frequent 1-itemsets")
    
    k = 2
    while True:
        prev_freq = freq_itemsets.get(k-1, [])
        if not prev_freq:
            break
        
        # Generate candidate k-itemsets
        candidates = []
        len_prev = len(prev_freq)
        for i in range(len_prev):
            for j in range(i+1, len_prev):
                union = prev_freq[i].union(prev_freq[j])
                if len(union) == k:
                    # Prune step: check if all (k-1)-subsets are frequent
                    subsets = [union - frozenset([item]) for item in union]
                    if all(subset in freq_itemsets[k-1] for subset in subsets):
                        candidates.append(union)
        
        # Remove duplicates
        candidates = list(set(candidates))
        if not candidates:
            break
        
        # Convert candidates to binary mask
        candidate_masks = np.zeros((len(candidates), num_items), dtype=int)
        for idx, candidate in enumerate(candidates):
            for item in candidate:
                candidate_masks[idx, item_index[item]] = 1
        
        # Optimized Frequency Counting using Matrix Multiplication
        # Compute the dot product between binary_matrix and candidate_masks.T
        # This gives the count of matching items for each candidate in each basket
        dot_product = binary_matrix @ candidate_masks.T  # Shape: (num_baskets, num_candidates)
        
        # Check where the count equals the size of the itemset (k)
        subset_check = dot_product == k  # Boolean array
        
        # Sum across baskets to get frequency counts for each candidate
        freq_counts = np.sum(subset_check, axis=0)
        
        # Select candidates that meet or exceed minsup
        freq_candidates_indices = np.where(freq_counts >= minsup)[0]
        
        # Store frequent k-itemsets
        freq_k = [frozenset([unique_items[idx] for idx in np.where(candidate_masks[i])[0]]) 
                  for i in freq_candidates_indices]
        if freq_k:
            freq_itemsets[k] = freq_k
            print(f"Generated {len(freq_k)} frequent {k}-itemsets")
            k += 1
        else:
            break
    
    return freq_itemsets

# Example Usage
if __name__ == "__main__":
    # Example binary matrix and unique items
    
    unique_items = sorted(set(item for basket in baskets for item in basket))
    item_index = {item: idx for idx, item in enumerate(unique_items)}
    
    # Create binary matrix
    binary_matrix = np.zeros((len(baskets), len(unique_items)), dtype=int)
    for i, basket in enumerate(baskets):
        for item in basket:
            binary_matrix[i, item_index[item]] = 1
    
    # Apriori Settings
    minsup = 1100  # Minimum support threshold
    
    # Generate frequent itemsets
    freq_itemsets = generate_freq_itemsets(binary_matrix, unique_items, minsup)
    

Generated 46 frequent 1-itemsets
Generated 430 frequent 2-itemsets
Generated 1783 frequent 3-itemsets
Generated 3425 frequent 4-itemsets
Generated 3290 frequent 5-itemsets
Generated 1766 frequent 6-itemsets
Generated 391 frequent 7-itemsets
Generated 31 frequent 8-itemsets


In [ ]:
import numpy as np
from itertools import chain, combinations

def generate_association_rules(freq_itemsets, binary_matrix, unique_items, minsup, minconf):
    """
    Generate association rules from frequent itemsets.

    Parameters:
    - freq_itemsets: dict, frequent itemsets with their sizes as keys
    - binary_matrix: np.ndarray, binary representation of baskets
    - unique_items: list, sorted list of unique items
    - minsup: int, minimum support threshold
    - minconf: float, minimum confidence threshold (e.g., 0.7 for 70%)

    Returns:
    - rules: list of tuples, each tuple contains (antecedent, consequent, support, confidence)
    """
    rules = []
    item_index = {item: idx for idx, item in enumerate(unique_items)}
    
    # Precompute support counts for all frequent itemsets
    support_counts = {}
    for k, itemsets in freq_itemsets.items():
        for itemset in itemsets:
            support_counts[itemset] = np.sum(
                np.all(binary_matrix[:, list(item_index[item] for item in itemset)] == 1, axis=1)
            )
    
    for k in freq_itemsets:
        if k < 2:
            continue  # Need at least 2 items to form a rule
        print(f"Generating rules from {k}-itemsets")
        for itemset in freq_itemsets[k]:
            # Generate all non-empty proper subsets of the itemset
            subsets = list(chain.from_iterable(combinations(itemset, r) for r in range(1, len(itemset))))
            for antecedent in subsets:
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                if not consequent:
                    continue
                # Calculate support and confidence
                support = support_counts[itemset]
                antecedent_support = support_counts.get(antecedent, 0)
                if antecedent_support == 0:
                    continue
                confidence = support / antecedent_support
                if confidence >= minconf:
                    rules.append((set(antecedent), set(consequent), support, confidence))
    
    return rules


In [23]:
rus = generate_association_rules(freq_itemsets, binary_matrix, unique_items, minsup, 0.8)

Generating rules from 2-itemsets
Generating rules from 3-itemsets
Generating rules from 4-itemsets
Generating rules from 5-itemsets
Generating rules from 6-itemsets
Generating rules from 7-itemsets
Generating rules from 8-itemsets


In [24]:
for antecedent, consequent, support, confidence in rus:
    print(f"Rule: {antecedent} -> {consequent} (Support: {support}, Confidence: {confidence:.2f})")

Rule: {40} -> {48} (Support: 1540, Confidence: 0.82)
Rule: {9} -> {48} (Support: 1133, Confidence: 0.85)
Rule: {13} -> {81} (Support: 1100, Confidence: 0.81)
Rule: {34} -> {48} (Support: 1329, Confidence: 0.80)
Rule: {13} -> {79} (Support: 1131, Confidence: 0.83)
Rule: {33} -> {51} (Support: 1703, Confidence: 0.87)
Rule: {70} -> {48} (Support: 1547, Confidence: 0.82)
Rule: {33} -> {48} (Support: 1681, Confidence: 0.86)
Rule: {13} -> {48} (Support: 1157, Confidence: 0.85)
Rule: {57} -> {59} (Support: 1591, Confidence: 0.81)
Rule: {36} -> {48} (Support: 2067, Confidence: 0.82)
Rule: {33} -> {69} (Support: 1634, Confidence: 0.83)
Rule: {57} -> {79} (Support: 1604, Confidence: 0.82)
Rule: {13} -> {69} (Support: 1177, Confidence: 0.87)
Rule: {57} -> {48} (Support: 1645, Confidence: 0.84)
Rule: {40} -> {69} (Support: 1565, Confidence: 0.84)
Rule: {36} -> {66} (Support: 2173, Confidence: 0.86)
Rule: {51} -> {48} (Support: 2972, Confidence: 0.80)
Rule: {33} -> {79} (Support: 1568, Confidence: 